<a href="https://colab.research.google.com/github/mmfara/Disparate-Impact-Remover-Enhanced/blob/main/MPA_Compas_DIR_tested_race_gender_separate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
!pip install aif360
!pip install aif360[LawSchoolGPA]
!pip install aif360[Reductions]
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
!pip install BlackBoxAuditing

pip install 'aif360[inFairness]'


In [2]:
# Import necessary libraries
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from aif360.algorithms.inprocessing import ExponentiatedGradientReduction
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.metrics import ClassificationMetric

#STARTING OFF

In [3]:
data = pd.read_csv("/content/COMPAS_race_gender_testing.csv")

In [4]:
data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,decile_score,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
0,1,1,0,0,1,0,3,1,0,0,0
1,1,1,1,0,1,4,4,1,0,0,1
2,1,0,0,1,1,14,6,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,1,0,4,0,0,0,0


In [5]:
X = data.drop(['two_year_recid'], axis =1)
y = data[['two_year_recid']]

In [6]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle =True, test_size=0.3, random_state=42)

In [7]:
print(X_train.shape)
print(X_test.shape)

(3694, 10)
(1584, 10)


In [8]:
# Create training DataFrame
train_df = X_train.copy()
train_df['two_year_recid'] = y_train

test_df = X_test.copy()
test_df['two_year_recid'] = y_test

In [9]:
# Group by race and gender, then label
group_label_counts = train_df.groupby(['race', 'sex', 'two_year_recid']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts)


          Negative (0)  Positive (1)
race sex                            
0    0             212           114
     1             681           459
1    0             250           141
     1             813          1024


In [10]:
# Check value counts by group and label
group_label_counts_test = test_df.groupby(['race', 'sex', 'two_year_recid']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts_test.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts_test)


          Negative (0)  Positive (1)
race sex                            
0    0             100            56
     1             288           193
1    0              96            62
     1             355           434


In [11]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler, SMOTE

# Define target and protected attribute columns
target_col = 'two_year_recid'
protected_cols = ['race', 'sex']  # Intersectional attributes

# Add label back to training data temporarily
X_train_temp = X_train.copy()
X_train_temp[target_col] = y_train

# Create a composite group identifier
X_train_temp['group'] = X_train_temp[protected_cols].astype(str).agg('_'.join, axis=1)

# Pick a sampler
sampler = RandomOverSampler(random_state=42)
# sampler = SMOTE(random_state=42)

resampled_X = []
resampled_y = []

# Oversample within each intersectional group
for group in X_train_temp['group'].unique():
    group_data = X_train_temp[X_train_temp['group'] == group]

    Xg = group_data.drop(columns=[target_col])
    yg = group_data[target_col]

    # Drop group before resampling
    Xg_no_group = Xg.drop(columns=['group'], errors='ignore')

    if yg.nunique() < 2:
        print(f"Skipping group {group}: only one class present.")
        continue

    Xg_res, yg_res = sampler.fit_resample(Xg_no_group, yg)

    # Restore the group column after resampling
    Xg_res['group'] = group
    resampled_X.append(Xg_res)
    resampled_y.append(pd.Series(yg_res))

# Combine everything
X_train_bal = pd.concat(resampled_X).reset_index(drop=True)
y_train_bal = pd.concat(resampled_y).reset_index(drop=True)

# Reattach target label
X_train_bal[target_col] = y_train_bal

# ✅ View final balanced counts per intersectional group
print("✅ Final balanced class counts per (race, sex):")
X_train_bal[['group', target_col]].groupby(['group', target_col]).size().unstack(fill_value=0)


✅ Final balanced class counts per (race, sex):


two_year_recid,0,1
group,,
0_0,212,212
0_1,681,681
1_0,250,250
1_1,1024,1024


In [12]:
X_train_bal.drop(columns=['group'], inplace=True)

In [13]:
dataset_train = X_train_bal.reset_index(drop=True)
dataset_test = test_df.reset_index(drop=True)

In [14]:
dataset_train.shape

(4334, 11)

In [15]:
dataset_test.shape

(1584, 11)

Checking my datasets

In [16]:
dataset_train

,c_charge_degree,race,age_cat,score_text,sex,priors_count,decile_score,juv_fel_count,juv_misd_count,juv_other_count,two_year_recid
0,0,0,1,1,1,0,5,0,0,0,0
1,1,0,0,0,1,2,1,0,0,0,0
2,1,0,0,2,1,10,9,0,0,0,0
3,1,0,2,0,1,0,1,0,0,0,0
4,1,0,1,2,1,0,8,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
4329,1,1,0,2,0,3,9,0,0,0,1
4330,1,1,0,1,0,1,6,0,0,0,1
4331,1,1,0,2,0,14,10,0,0,0,1
4332,1,1,0,0,0,2,3,0,0,0,1


In [17]:
# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

privileged_groups = [{'race': 1, 'sex': 1}]

unprivileged_groups = [
    {'race': r, 'sex': s}
    for r in data['race'].unique()
    for s in data['sex'].unique()
    if not (r == 1 and s == 1)]


#DisparateImpactRemover

In [18]:
import numpy as np
import logging
from itertools import product
from aif360.algorithms import Transformer


class DisparateImpactRemover(Transformer):
    """
    Extended Disparate Impact Remover:
    - Uses GeneralRepairer for rank computation (faithful to Feldman et al.)
    - Supports intersectional groups
    - Repairs feature values toward a shared global distribution
    - Preserves within-group rank
    """

    def __init__(self, repair_level=1.0, sensitive_attribute=None, min_group_size=20, verbose=True):
        super().__init__()
        from BlackBoxAuditing.repairers.GeneralRepairer import Repairer
        self.Repairer = Repairer

        self.repair_level = repair_level
        self.min_group_size = min_group_size
        self.verbose = verbose

        if not 0.0 <= repair_level <= 1.0:
            raise ValueError("'repair_level' must be between 0.0 and 1.0.")

        if isinstance(sensitive_attribute, str):
            self.sensitive_attributes = [sensitive_attribute]
        elif isinstance(sensitive_attribute, list):
            self.sensitive_attributes = sensitive_attribute
        elif sensitive_attribute is None:
            self.sensitive_attributes = []
        else:
            raise TypeError("sensitive_attribute must be str, list, or None")

        if self.verbose:
            logging.basicConfig(level=logging.INFO)

    def fit_transform(self, dataset):
        if not self.sensitive_attributes:
            self.sensitive_attributes = dataset.protected_attribute_names[:1]

        indices = [dataset.feature_names.index(attr) for attr in self.sensitive_attributes]
        protected_values = [np.unique(dataset.features[:, idx]) for idx in indices]
        group_combinations = list(product(*protected_values))

        repaired = dataset.copy()
        all_features = repaired.features.copy()

        feature_dim = all_features.shape[1]
        protected_set = set(indices)
        non_protected_indices = [i for i in range(feature_dim) if i not in protected_set]

        # Step 1: Collect data for global target distribution
        pooled_values = []
        group_indices = {}

        for group_vals in group_combinations:
            mask = np.logical_and.reduce([
                all_features[:, idx] == val for idx, val in zip(indices, group_vals)
            ])
            group_features = all_features[mask][:, non_protected_indices]

            if group_features.shape[0] < self.min_group_size:
                if self.verbose:
                    logging.warning(f"Skipping group {group_vals} (size={group_features.shape[0]})")
                continue

            pooled_values.append(group_features)
            group_indices[group_vals] = np.where(mask)[0]

        if not pooled_values:
            raise ValueError("No eligible groups met the minimum size requirement.")

        # Build global sorted target distribution
        pooled_array = np.vstack(pooled_values)
        global_sorted = np.sort(pooled_array, axis=0)

        # Step 2: Repair each group by aligning its ranks to global distribution
        for group_vals, idxs in group_indices.items():
            group_data = all_features[idxs][:, non_protected_indices]
            n = len(idxs)

            # Use GeneralRepairer to compute feature-wise ranks
            repairer = self.Repairer(group_data.tolist(), 0, 1.0, False)
            ranked = np.array(repairer.repair(group_data.tolist()))

            # Use the ranks to align to global distribution
            ranks = np.argsort(np.argsort(group_data, axis=0))
            aligned_by_rank = np.zeros_like(group_data)

            for col in range(group_data.shape[1]):
                aligned_by_rank[:, col] = global_sorted[ranks[:, col], col]

            # Interpolate: original + global target
            group_repaired = (
                (1 - self.repair_level) * group_data +
                self.repair_level * aligned_by_rank
            )

            all_features[idxs[:, None], non_protected_indices] = group_repaired

            if self.verbose:
                logging.info(f"Repaired group {group_vals} (size={n}) toward global target.")

        # Restore protected attributes
        for idx in indices:
            all_features[:, idx] = dataset.features[:, idx]

        repaired.features = all_features
        return repaired


In [19]:
# Applying DisparateImpactRemover
dir = DisparateImpactRemover(
    repair_level=0.4,
    sensitive_attribute=['race', 'sex'],
    min_group_size=502,
    verbose=True
)

In [20]:
dataset_train_bld = BinaryLabelDataset(df=dataset_train,
                             label_names=['two_year_recid'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=['race', 'sex'],
                             privileged_protected_attributes=privileged_groups)


dataset_test_bld = BinaryLabelDataset(df=dataset_test,
                                      label_names=['two_year_recid'],
                                      favorable_label=favorable_label,
                                      unfavorable_label=unfavorable_label,
                                      protected_attribute_names=['race', 'sex'],
                                      privileged_protected_attributes=privileged_groups)

###Fitting disparate impact removal on the dataset

In [21]:
# Fit and transform the training data
dir_dataset = dir.fit_transform(dataset_train_bld)

In [22]:
#Disparate Impact Remover transforms dataset
dir_dataset

               instance weights        features                              \
                                                protected attribute           
                                c_charge_degree                race age_cat   
instance names                                                                
0                           1.0             0.0                 0.0     0.6   
1                           1.0             0.6                 0.0     0.0   
2                           1.0             0.6                 0.0     0.0   
3                           1.0             0.6                 0.0     1.2   
4                           1.0             0.6                 0.0     0.6   
...                         ...             ...                 ...     ...   
4329                        1.0             1.0                 1.0     0.0   
4330                        1.0             1.0                 1.0     0.0   
4331                        1.0             1.0     

In [23]:
# Define features and target variable
X_train = dir_dataset.features
y_train = dir_dataset.labels.ravel()
X_test = dataset_test_bld.features
y_test = dataset_test_bld.labels.ravel()

In [24]:
X_train

array([[0. , 0. , 0.6, ..., 0. , 0. , 0. ],
       [0.6, 0. , 0. , ..., 0. , 0. , 0. ],
       [0.6, 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [1. , 1. , 0. , ..., 0. , 0. , 0. ],
       [1. , 1. , 0. , ..., 0. , 0. , 0. ],
       [1. , 1. , 1. , ..., 0. , 0. , 0. ]])

In [25]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)

X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

y_train = pd.DataFrame(y_train, columns = y.columns)

y_test = pd.DataFrame(y_test, columns = y.columns)

In [27]:
dataset_train = pd.concat([X_train_scaled, y_train], axis=1)

dataset_test = pd.concat([X_test_scaled, y_test], axis=1)

#Model

In [28]:
model = LogisticRegression()

In [29]:
model.fit(X_train_scaled, y_train['two_year_recid'])

LogisticRegression()

In [30]:
y_pred = model.predict(X_test_scaled)

In [31]:
accuracy_score(y_pred, y_test)

0.6609848484848485

In [32]:
y_pred

array([1., 0., 0., ..., 0., 1., 1.])

In [33]:
y_pred = pd.DataFrame(y_pred, columns=['two_year_recid'])

In [34]:
X_test = pd.DataFrame(X_test, columns=X.columns)

In [35]:
dataset_test = pd.concat([X_test, y_test], axis=1)

dataset_testpred = pd.concat([X_test, y_pred], axis=1)

In [36]:
# Convert DataFrames back to AIF360 BinaryLabelDataset
dataset_test = BinaryLabelDataset(df=dataset_test,
                                          label_names=['two_year_recid'],
                                          favorable_label=favorable_label,
                                          unfavorable_label=unfavorable_label,
                                          protected_attribute_names=['race', 'sex'],
                                          privileged_protected_attributes=privileged_groups)

dataset_testpred = BinaryLabelDataset(df=dataset_testpred,
                                         label_names=['two_year_recid'],
                                         favorable_label=favorable_label,
                                         unfavorable_label=unfavorable_label,
                                         protected_attribute_names=['race', 'sex'],
                                         privileged_protected_attributes=privileged_groups)

In [37]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test,
    dataset_testpred,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [38]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.660985
Disparate Impact: 0.761563
Mean Difference: -0.161073


In [39]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [40]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: -0.1610732477222182
Disparate Impact: 0.7615632411766788
Equal Opportunity Difference: -0.13817475958332714


In [41]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = -0.138175
Difference in False Positive Rates (Unprivileged - Privileged) = -0.086497


In [42]:
import pandas as pd
import numpy as np
from itertools import product
from aif360.metrics import ClassificationMetric

# Assumes you defined this earlier:
# privileged_groups = [{'race': 1, 'sex': 1}]

# Get indices of protected attributes
race_idx = dataset_test.feature_names.index('race')
sex_idx = dataset_test.feature_names.index('sex')

# All intersectional (race, sex) combinations
race_vals = np.unique(dataset_test.features[:, race_idx])
sex_vals = np.unique(dataset_test.features[:, sex_idx])
group_combinations = list(product(race_vals, sex_vals))

# Collect results
results = []

for (race_val, sex_val) in group_combinations:
    current_group = {'race': race_val, 'sex': sex_val}

    if current_group in privileged_groups:
        continue  # Skip privileged group

    metric = ClassificationMetric(
        dataset_test,
        dataset_testpred,
        unprivileged_groups=[current_group],
        privileged_groups=privileged_groups
    )

    # Manual subgroup accuracy
    mask = (
        (dataset_test.features[:, race_idx] == race_val) &
        (dataset_test.features[:, sex_idx] == sex_val)
    )
    y_true = dataset_test.labels[mask]
    y_pred = dataset_testpred.labels[mask]
    accuracy = (y_true == y_pred).mean()

    results.append({
        'race': int(race_val),
        'sex': int(sex_val),
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        'Predictive Equality Difference': metric.false_positive_rate_difference(),
        'Subgroup Accuracy': round(accuracy, 6)
    })

# Results table
df_results = pd.DataFrame(results)
display(df_results)


,race,sex,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Subgroup Accuracy
0,0,0,-0.239641,-0.150922,0.645259,-0.187324,0.692308
1,0,1,-0.105892,-0.109286,0.843248,-0.018574,0.607069
2,1,0,-0.251488,-0.216590,0.627722,-0.185241,0.664557


#DIR+EO

In [43]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

# Initialize EqualizedOdds
eo = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups, seed = 101)

# Fit the EqualizedOdds model
test_predictions = eo.fit_predict(dataset_test, dataset_testpred)

In [44]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [45]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.644571
Disparate Impact: 0.916964
Mean Difference: -0.047043


In [46]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [47]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: -0.047043068608460636
Disparate Impact: 0.9169642480266769
Equal Opportunity Difference: -0.008231214900647488


In [48]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = -0.008231
Difference in False Positive Rates (Unprivileged - Privileged) = 0.007025


In [49]:
import pandas as pd
import numpy as np
from itertools import product
from aif360.metrics import ClassificationMetric

# Assumes you defined this earlier:
# privileged_groups = [{'race': 1, 'sex': 1}]

# Get indices of protected attributes
race_idx = dataset_test.feature_names.index('race')
sex_idx = dataset_test.feature_names.index('sex')

# All intersectional (race, sex) combinations
race_vals = np.unique(dataset_test.features[:, race_idx])
sex_vals = np.unique(dataset_test.features[:, sex_idx])
group_combinations = list(product(race_vals, sex_vals))

# Collect results
results = []

for (race_val, sex_val) in group_combinations:
    current_group = {'race': race_val, 'sex': sex_val}

    if current_group in privileged_groups:
        continue  # Skip privileged group

    metric = ClassificationMetric(
        dataset_test_bld,
        test_predictions,
        unprivileged_groups=[current_group],
        privileged_groups=privileged_groups
    )

    # Manual subgroup accuracy
    mask = (
        (dataset_test.features[:, race_idx] == race_val) &
        (dataset_test.features[:, sex_idx] == sex_val)
    )
    y_true = dataset_test_bld.labels[mask]
    y_pred = test_predictions.labels[mask]
    accuracy = (y_true == y_pred).mean()

    results.append({
        'race': int(race_val),
        'sex': int(sex_val),
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        'Predictive Equality Difference': metric.false_positive_rate_difference(),
        'Subgroup Accuracy': round(accuracy, 6)
    })

# Results table
df_results = pd.DataFrame(results)
display(df_results)


,race,sex,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Subgroup Accuracy
0,0,0,-0.124232,-0.024194,0.780718,-0.090000,0.685897
1,0,1,0.009344,0.022624,1.016492,0.075694,0.604990
2,1,0,-0.142489,-0.089862,0.748492,-0.097917,0.664557


#DIR+ROC

In [50]:
# Apply Reject Option Classification
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups, metric_name="Average odds difference",
                                 low_class_thresh=0.3, high_class_thresh=0.8, num_class_thresh=100, num_ROC_margin=50,
                                 metric_ub=0.05, metric_lb=-0.05)

In [ ]:
test_predictions = ROC.fit_predict(dataset_test, dataset_testpred)

In [52]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [53]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.660985
Disparate Impact: 0.761563
Mean Difference: -0.161073


In [54]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [55]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: -0.1610732477222182
Disparate Impact: 0.7615632411766788
Equal Opportunity Difference: -0.13817475958332714


In [56]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = -0.138175
Difference in False Positive Rates (Unprivileged - Privileged) = -0.086497


In [57]:
import pandas as pd
import numpy as np
from itertools import product
from aif360.metrics import ClassificationMetric

# Assumes you defined this earlier:
# privileged_groups = [{'race': 1, 'sex': 1}]

# Get indices of protected attributes
race_idx = dataset_test.feature_names.index('race')
sex_idx = dataset_test.feature_names.index('sex')

# All intersectional (race, sex) combinations
race_vals = np.unique(dataset_test.features[:, race_idx])
sex_vals = np.unique(dataset_test.features[:, sex_idx])
group_combinations = list(product(race_vals, sex_vals))

# Collect results
results = []

for (race_val, sex_val) in group_combinations:
    current_group = {'race': race_val, 'sex': sex_val}

    if current_group in privileged_groups:
        continue  # Skip privileged group

    metric = ClassificationMetric(
        dataset_test_bld,
        test_predictions,
        unprivileged_groups=[current_group],
        privileged_groups=privileged_groups
    )

    # Manual subgroup accuracy
    mask = (
        (dataset_test.features[:, race_idx] == race_val) &
        (dataset_test.features[:, sex_idx] == sex_val)
    )
    y_true = dataset_test_bld.labels[mask]
    y_pred = test_predictions.labels[mask]
    accuracy = (y_true == y_pred).mean()

    results.append({
        'race': int(race_val),
        'sex': int(sex_val),
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        'Predictive Equality Difference': metric.false_positive_rate_difference(),
        'Subgroup Accuracy': round(accuracy, 6)
    })

# Results table
df_results = pd.DataFrame(results)
display(df_results)


,race,sex,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Subgroup Accuracy
0,0,0,-0.239641,-0.150922,0.645259,-0.187324,0.692308
1,0,1,-0.105892,-0.109286,0.843248,-0.018574,0.607069
2,1,0,-0.251488,-0.216590,0.627722,-0.185241,0.664557
